In [1]:

%load_ext autoreload
%autoreload 2

In [2]:
import glob

import pandas as pd

In [3]:
## delete the folders not in live config ( extra crap )
live_config_files = glob.glob('cfgs_live/*', recursive=True)
allowed_symbols = []
symbol_files = {}
for f in live_config_files:
    symbol = f.split('\\')[-1]
    symbol = symbol.replace('.json', 'USDT')
    allowed_symbols.append(symbol)
    symbol_files[symbol] = f
symbol_files
#allowed_symbols = ["ALICEUSDT","XRPUSDT","HOTUSDT"]

{'1000XECUSDT': 'cfgs_live\\1000XEC.json',
 '1INCHUSDT': 'cfgs_live\\1INCH.json',
 'ADAUSDT': 'cfgs_live\\ADA.json',
 'AKROUSDT': 'cfgs_live\\AKRO.json',
 'ALGOUSDT': 'cfgs_live\\ALGO.json',
 'ALICEUSDT': 'cfgs_live\\ALICE.json',
 'ALPHAUSDT': 'cfgs_live\\ALPHA.json',
 'ANKRUSDT': 'cfgs_live\\ANKR.json',
 'API3USDT': 'cfgs_live\\API3.json',
 'ARUSDT': 'cfgs_live\\AR.json',
 'ATOMUSDT': 'cfgs_live\\ATOM.json',
 'BAKEUSDT': 'cfgs_live\\BAKE.json',
 'BANDUSDT': 'cfgs_live\\BAND.json',
 'BELUSDT': 'cfgs_live\\BEL.json',
 'BLZUSDT': 'cfgs_live\\BLZ.json',
 'BTSUSDT': 'cfgs_live\\BTS.json',
 'C98USDT': 'cfgs_live\\C98.json',
 'CHZUSDT': 'cfgs_live\\CHZ.json',
 'COMPUSDT': 'cfgs_live\\COMP.json',
 'COTIUSDT': 'cfgs_live\\COTI.json',
 'CRVUSDT': 'cfgs_live\\CRV.json',
 'DENTUSDT': 'cfgs_live\\DENT.json',
 'DGBUSDT': 'cfgs_live\\DGB.json',
 'DODOUSDT': 'cfgs_live\\DODO.json',
 'DOGEUSDT': 'cfgs_live\\DOGE.json',
 'DUSKUSDT': 'cfgs_live\\DUSK.json',
 'ENJUSDT': 'cfgs_live\\ENJ.json',
 'ETCUSDT':

In [4]:
# ## NOW copy result folders :)
# import json
# import shutil
#
# files = glob.iglob('backtests/binance/' + '**/live_config.json', recursive=True)
#
# for filename in files:
#     with open(filename) as f:
#         config = json.load(f)
#     symbol = config['config_name'].split('_')[2]
#
#     if symbol not in symbol_files: continue
#
#     with open(symbol_files[symbol]) as f:
#         production_config = json.load(f)
#
#     long_pc = production_config['long']
#     long_c = config['long']
#
#     k_pc = long_pc.keys()
#     k_c = long_c.keys()
#
#     if len(k_c) != len(k_c): continue
#
#     same = True
#     for k in k_pc:
#         if k not in k_c:
#             same = False
#             break
#         v_pc = long_pc[k]
#         v_c = long_c[k]
#         #print(f'{k}={v_c},{v_pc}')
#
#     if not same: continue
#
#     print(f'{filename} is same as {symbol_files[symbol]}')
#
#     src = filename.replace('\\live_config.json', '')
#     dst = f'backtests/binance_production/{symbol}'
#     shutil.copytree(src=src, dst=dst, dirs_exist_ok=True)
#


In [5]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance_production/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/binance_production','')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())

        #d['adgt_%'] = d['adgl_%']+d['adgs_%']
        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_284248/891417201.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_284248/891417201.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_284248/891417201.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_284248/891417201.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_inde

,file,v,s,long,tgl_%,adgl_%,bnkrl_%,ebrl,short,tgs_%,adgs_%,bnkrs_%,ebrs
0,\1000XECUSDT\backtest_result.txt,5.5,1000XECUSDT,True,8.699,0.0210,100.00,0.9114,True,39.490,0.1260,100.0,0.8667
1,\1INCHUSDT\backtest_result.txt,5.5,1INCHUSDT,True,15.600,0.0512,100.00,0.8765,True,30.010,0.0896,100.0,0.6393
2,\ADAUSDT\backtest_result.txt,5.5,ADAUSDT,True,12.870,0.0248,100.00,0.8961,True,21.780,0.0495,100.0,0.7763
3,\AKROUSDT\backtest_result.txt,5.5,AKROUSDT,True,22.200,0.0709,100.00,0.9258,True,8.130,0.0260,100.0,0.8021
4,\ALGOUSDT\backtest_result.txt,5.5,ALGOUSDT,True,25.110,0.0811,100.00,0.9135,True,8.388,0.0203,100.0,0.7959
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,\XMRUSDT\backtest_result.txt,5.5,XMRUSDT,True,4.557,0.0178,100.00,0.8897,True,16.830,0.0344,100.0,0.8464
75,\XRPUSDT\backtest_result.txt,5.5,XRPUSDT,True,-50.600,-1.6800,5.99,0.8771,True,3.811,-0.0122,100.0,0.6800
76,\ZENUSDT\backtest_result.txt,5.5,ZENUSDT,True,41.930,0.1300,100.00,0.8677,True,12.430,0.0286,100.0,0.7565
77,\ZILUSDT\backtest_result.txt,5.5,ZILUSDT,True,3.347,0.0125,100.00,0.8991,True,16.280,-0.0436,100.0,0.6366


Now filter the dataframe

In [6]:
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
              # & (big_data['adgt_%'] > 0.1)
              # & (big_data['adgl_%'] > 0.04)
              # & (big_data['adgs_%'] > 0.04)
              # & (big_data['bnkrl_%'] > 99.0)
              # & (big_data['bnkrs_%'] > 99.0)
              # & (big_data['ebrs'] > .9)
              # #& (big_data['ebrl'] > .9)
              # & (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
              ]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,long,tgl_%,adgl_%,bnkrl_%,ebrl,short,tgs_%,adgs_%,bnkrs_%,ebrs,adgt_%
0,\1000XECUSDT\backtest_result.txt,5.5,1000XECUSDT,True,8.699,0.0210,100.00,0.9114,True,39.490,0.1260,100.0,0.8667,0.1470
1,\1INCHUSDT\backtest_result.txt,5.5,1INCHUSDT,True,15.600,0.0512,100.00,0.8765,True,30.010,0.0896,100.0,0.6393,0.1408
2,\ADAUSDT\backtest_result.txt,5.5,ADAUSDT,True,12.870,0.0248,100.00,0.8961,True,21.780,0.0495,100.0,0.7763,0.0743
3,\AKROUSDT\backtest_result.txt,5.5,AKROUSDT,True,22.200,0.0709,100.00,0.9258,True,8.130,0.0260,100.0,0.8021,0.0969
4,\ALGOUSDT\backtest_result.txt,5.5,ALGOUSDT,True,25.110,0.0811,100.00,0.9135,True,8.388,0.0203,100.0,0.7959,0.1014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,\XMRUSDT\backtest_result.txt,5.5,XMRUSDT,True,4.557,0.0178,100.00,0.8897,True,16.830,0.0344,100.0,0.8464,0.0522
75,\XRPUSDT\backtest_result.txt,5.5,XRPUSDT,True,-50.600,-1.6800,5.99,0.8771,True,3.811,-0.0122,100.0,0.6800,-1.6922
76,\ZENUSDT\backtest_result.txt,5.5,ZENUSDT,True,41.930,0.1300,100.00,0.8677,True,12.430,0.0286,100.0,0.7565,0.1586
77,\ZILUSDT\backtest_result.txt,5.5,ZILUSDT,True,3.347,0.0125,100.00,0.8991,True,16.280,-0.0436,100.0,0.6366,-0.0311


In [7]:
df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',
                   inplace=True)
dx = df[['file','s','adgt_%','adgl_%','adgs_%','bnkrl_%', 'bnkrs_%','ebrl', 'ebrs']]
dx

,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs
0,\1000XECUSDT\backtest_result.txt,1000XECUSDT,0.1470,0.0210,0.1260,100.00,100.0,0.9114,0.8667
1,\1INCHUSDT\backtest_result.txt,1INCHUSDT,0.1408,0.0512,0.0896,100.00,100.0,0.8765,0.6393
2,\ADAUSDT\backtest_result.txt,ADAUSDT,0.0743,0.0248,0.0495,100.00,100.0,0.8961,0.7763
3,\AKROUSDT\backtest_result.txt,AKROUSDT,0.0969,0.0709,0.0260,100.00,100.0,0.9258,0.8021
4,\ALGOUSDT\backtest_result.txt,ALGOUSDT,0.1014,0.0811,0.0203,100.00,100.0,0.9135,0.7959
...,...,...,...,...,...,...,...,...,...
74,\XMRUSDT\backtest_result.txt,XMRUSDT,0.0522,0.0178,0.0344,100.00,100.0,0.8897,0.8464
75,\XRPUSDT\backtest_result.txt,XRPUSDT,-1.6922,-1.6800,-0.0122,5.99,100.0,0.8771,0.6800
76,\ZENUSDT\backtest_result.txt,ZENUSDT,0.1586,0.1300,0.0286,100.00,100.0,0.8677,0.7565
77,\ZILUSDT\backtest_result.txt,ZILUSDT,-0.0311,0.0125,-0.0436,100.00,100.0,0.8991,0.6366
